# Using LLMs in Python

In [1]:
import os

import dotenv
import PIL.Image

import google.generativeai as genai

In [2]:
loaded: bool = dotenv.load_dotenv()
print("Loaded environment variables." if loaded else "Could not load environment variables.")

Loaded environment variables.


In [3]:
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

In [4]:
model_type = 'models/gemini-1.5-flash'
model = genai.GenerativeModel(model_type)
model_info = genai.get_model(model_type)
print(f"Model name: {model_info.name}\nModel version: {model_info.version}\nInput token limit: {model_info.input_token_limit}\nOutput token limit: {model_info.output_token_limit}")

Model name: models/gemini-1.5-flash
Model version: 001
Input token limit: 1000000
Output token limit: 8192


In [5]:
# Counting tokens
prompt = "I want to design a building facade that helps with carbon-capture and fits in with the aesthetic of central London."
model.count_tokens(prompt)

total_tokens: 23

In [6]:
# Prompting
response = model.generate_content(prompt)
print(response.text)

##  A Central London Facade for Carbon Capture:

Here are some ideas for a building facade in central London that integrates carbon capture and fits seamlessly with the existing aesthetic:

**1. "Green Wall" Facade:**

* **Aesthetic:** Mimic traditional London brickwork with variations in colour and texture to create depth. Integrate vertical gardens with native plants and vines.
* **Carbon Capture:** Use biofilters within the green wall system to absorb CO2. The plants act as living filters, while specialized materials and microorganisms in the filters further enhance carbon capture.
* **Integration:** Incorporate solar panels cleverly into the facade's design, perhaps hidden behind the plants or integrated into the brickwork.  
* **Example:** The "GreenPix" building in London, which incorporates a large green wall and solar panels, could be a source of inspiration.

**2. "Air Purifier" Facade:**

* **Aesthetic:** Use traditional materials like limestone or sandstone, but with a more 

### Using LangChain

[Langchain](https://python.langchain.com/docs/introduction/) is a framework for developing applications powered by large language models (LLMs). In this example, [ChatGoogleGenerativeAI](https://python.langchain.com/docs/integrations/chat/google_generative_ai/) is used to exemplify interaction with the same model.

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

**Note:** It is recommended to experiment with model hyperparameters, such as [temperature](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/adjust-parameter-values#temperature), to find the right configuration for your application. Temperature influences the degree of randomness or 'creativity' in the model's output. Lower values will lead to more deterministic responses, while higher values will result in more 'creative' responses.

In [43]:
model_temperature = 0.3
llm = ChatGoogleGenerativeAI(model=model_type, google_api_key=GEMINI_API_KEY, temperature=model_temperature)

In [9]:
response = llm.invoke(prompt)

In [10]:
print(response.content)

## Carbon-Capturing Facade for Central London: A Blend of Functionality and Aesthetics

Here's a concept for a building facade that combines carbon capture technology with the classic elegance of central London:

**Concept:**

* **Inspired by Victorian Architecture:** The facade draws inspiration from the iconic brickwork and intricate details of Victorian buildings in London. 
* **Integrated Bio-Wall:** A living wall system, featuring a carefully curated selection of carbon-absorbing plants, is incorporated into the facade. This bio-wall acts as a natural carbon sink, absorbing CO2 from the surrounding air.
* **Photovoltaic Panels:**  Hidden behind the brickwork, photovoltaic panels are strategically placed to capture solar energy. This energy powers the building and the bio-wall's lighting and irrigation systems.
* **Smart Ventilation System:** The facade features a smart ventilation system that optimizes airflow and maximizes the bio-wall's carbon capture efficiency. This system can

### RAG with LangChain

![Diagrams - RAG Overview](Diagrams%20-%20RAG%20Overview.jpg)

<i>Figure: Overview of RAG - using document data and the user query to generate an output.</i>

In [11]:
import pandas as pd

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

In [12]:
# Load and split a PDF file
document_path = "../resources/document_examples/The_Evolution_and_Change_of_Building_Facades_Research_D_Turgut_Cikis.pdf"
document_loader = PyPDFLoader(document_path)
document_pages = document_loader.load_and_split()
print(f"Loaded the specified document: {len(document_pages)} pages")

Loaded the specified document: 121 pages


In [13]:
# Visualzie the contents of the 5th page of the document
print(document_pages[20].page_content)

10 hung on the building structure, usually from floor to floor. It can provide a variety of 
exterior appearances but characterized by narrowly spaced vertical and horizontal caps 
with glass or metal infill panels. These systems provide a finished exterior appearance 
and most often a semi-finished interior as well (Quirouette 1982). They are also 
designed to accommodate structural deflections, control wind-driven rain and leakage, 
minimize the effects of solar radiation and provide for maintenance-free long term 
performance. Most of today's metal curtain wall systems are constructed of lightweight 
aluminum, although some may be of steel.  
 
2.2. Historical Background 
 
Even walls present in building traditions since the beginning of human 
existence, the intention of creating large openings on it was first seen in the gothic 
architecture.  But the concept of fully glazed wall developed fitfully soon after this 
period. Therefore it was an architectural idea only awaiting neces

In [14]:
# Create chunks from the loaded PDF pages
chunk_size=8000
chunk_overlap=1000

character_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
text_chunks = character_splitter.split_text("\n\n".join(str(page.page_content) for page in document_pages))
print(f"Created {len(text_chunks)} chunks of size {chunk_size} from the original {len(document_pages)} document pages.")

Created 32 chunks of size 8000 from the original 121 document pages.


In [15]:
# Set the model that will embed the text chunks
embeddings_model = "models/embedding-001"
embeddings = GoogleGenerativeAIEmbeddings(model=embeddings_model, google_api_key=GEMINI_API_KEY)

# Create a vectorstore from the text chunks, using the set model to embed them
number_of_chunks_to_retrieve_per_query = 3
vectorstore_retriever = Chroma.from_texts(texts=text_chunks, embedding=embeddings).as_retriever(search_kwargs={"k":number_of_chunks_to_retrieve_per_query})

In [85]:
# Create a chain for prompting, retrieving chunks from the vectorstore and using them in the llm to produce an answer
llm_qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore_retriever, return_source_documents=True)

In [17]:
prompt = {
    "query" : "I want to design a facade for a large residential building in a hot climate in Sourthern Europe. The aim is to maximize the energy efficiency of the building " + \
    "through the facade design while minimizing the weight of the facade. What type of cladding system should be used? What percentage of it should be glazed?"
}

In [18]:
response = llm_qa_chain.invoke(prompt)

In [19]:
print(response["result"])

Here's a breakdown of facade design considerations for a large residential building in a hot Southern European climate, focusing on energy efficiency and minimizing weight:

**Cladding System Considerations:**

* **Ventilated Facade:** This is a strong contender for a hot climate. It creates an air gap between the cladding and the building's insulation, allowing for natural ventilation and reducing heat gain. 
    * **Materials:**
        * **Lightweight Metal Panels:** Aluminum or thin-gauge steel panels are excellent choices. They offer durability, low weight, and can be easily fabricated into various shapes and sizes.
        * **High-Performance Concrete Panels:**  Consider lightweight concrete panels with insulation cores. They can provide excellent thermal performance and a modern aesthetic.
        * **Composite Panels:**  Combining materials like metal and wood or metal and fiber cement can offer a balance of aesthetics, weight, and performance.

* **Glazing:**
    * **High-Per

In [20]:
# The specific data from the document that was used to generate the prompt can be retrieved from the response
print("\n".join([document.page_content for document in response["source_documents"]]))

51 appearance. Zinc panels should be well in ventilated on the on the non-exposed 
surfaces. Water from condensation or leaking can dissolve the metal relatively quickly.  
C. Titanium is also a popular but costly material, which is rarely used in 
distinguished example of architecture. But because of the cost factor it doesn’t used 
commercially in industrialized cladding system. 
 
2.5.2. Transparent Curtain Walls 
 
The very first material evocation of curtain wall is glass. As it is known glass is 
one of the most common and also one of the oldest building materials. Its feature and 
material properties are developed during the course of history. Today it is possible to 
find different types of glass that is produced for specific needs in building market. But 
according to Wiggenton, The development in glass enormous but they are concentrated 
on finishes and coatings, and gases for insulated units, and small manipulations on float 
glass. Spurred by energy concerns, aesthetics and

### Returning structured output

In order to ensure that the model response is compatible with the continuation of the application pipeline, [a specific output structure can be imposed](https://python.langchain.com/docs/how_to/structured_output/) - such as Pydantic classes, JSON schemas or custom definitions.

In [75]:
from typing import Optional
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

In [69]:
# Create a Pydantic class to enforce the structure of the model's response
class FacadeSpecifications(BaseModel):
    """The facade specifications recommended by the LLM in accordance with the provided documentation."""
    
    cladding_system: Optional[str] = Field(description="The cladding system of the building.")
    glazing_system: Optional[str] = Field(description="The glazing system of the building.")
    predominant_material: Optional[str] = Field(description="The predominant cladding material in the facade.")
    glazing_percentage: Optional[int] = Field(description="The percentage of the facade that should be comprised of glazing.")

In [78]:
# Create a parser that will process the model outputs into the predefined BaseModel class
parser = PydanticOutputParser(pydantic_object=FacadeSpecifications)

In [81]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "The facade specifications recommended by the LLM in accordance with the provided documentation.", "properties": {"cladding_system": {"anyOf": [{"type": "string"}, {"type": "null"}], "description": "The cladding system of the building.", "title": "Cladding System"}, "glazing_system": {"anyOf": [{"type": "string"}, {"type": "null"}], "description": "The glazing system of the building.", "title": "Glazing System"}, "predominant_material": {"anyOf": [{"type": "string"}, {"type": "null"}], "description": "The predominant cladding m

In [79]:
# Create a prompt template to query the model
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(
            "Answer the user's question as best as possible with information about the cladding system, glazing system, materials and glazing percentage in the context of the discussed facade design." + \
            "\n{format_instructions}\n{question}"
        )
    ],
    input_variables=["question"],
    partial_variables={
        "format_instructions": parser.get_format_instructions(),
    }
)

In [99]:
# Define the actual query from the user
query = "I want to design a facade for a large residential building in a hot climate in Sourthern Europe. The aim is to maximize the energy efficiency of the building " + \
    "through the facade design while minimizing the weight of the facade. Consider how much glazing should be used. What type of cladding system should be used? What percentage of it should be glazed?"

# Use the prompt template and the user query to create the input to the model
input = prompt.format_prompt(question=query)

print(input.to_string())

Human: Answer the user's question as best as possible with information about the cladding system, glazing system, materials and glazing percentage in the context of the discussed facade design.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "The facade specifications recommended by the LLM in accordance with the provided documentation.", "properties": {"cladding_system": {"anyOf": [{"type": "string"}, {"type": "null"}], "description": "The cladding system of the building.", "title": "Cladding System"}, "glazing_system": {"anyOf": [{"type": "string"}, {"type": "null"}], "des

In [100]:
# Invoke the LLM chain with a string representation of the input
output = llm_qa_chain.invoke(input.to_string())

In [101]:
print(output["result"])

```json
{
"cladding_system": "Unit and Mullion System",
"glazing_system": "Outside Glazed Stick System",
"predominant_material": "Aluminum",
"glazing_percentage": 60
}
```


In [102]:
# parse the output into the predefined Pydantic class
parsed_output = parser.parse(output["result"])
print(parsed_output)

cladding_system='Unit and Mullion System' glazing_system='Outside Glazed Stick System' predominant_material='Aluminum' glazing_percentage=60


In [103]:
parsed_output.glazing_percentage

60